<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#Decision-Tree-Classifier" data-toc-modified-id="Decision-Tree-Classifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Decision Tree Classifier</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [43]:
import numpy as np
import pandas as pd
import re
import nltk

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline

from catboost import CatBoostClassifier

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import spacy

In [44]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.sample(15)
#data.head(15)

,Unnamed: 0,text,toxic
95508,95600,Hi Pete \n\nYou're a prick!\n\nYours sincerely...,1
17022,17039,"Clearly you *haven't* read WP:NPA, otherwise y...",0
29452,29491,""" \nIt's not so easy, I do it as good I can, b...",0
51017,51073,I didn't mean it. I apologise. 2.124.146.187,0
127197,127328,This has been discussed to death. The consens...,0
103154,103251,"""\n\n Now, for those who won’t read, let me tr...",0
96067,96159,Others prior to that time should be checked on...,0
132216,132352,to Chernobyl disaster,0
120810,120915,Teaching how to make pipe bombs. \n\nShame oin...,0
62619,62686,"""\n\nWhy should I give a fuck! LEAVE ME ALONE,...",1


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [46]:
display(data['toxic'].value_counts())

0    143106
1     16186
Name: toxic, dtype: int64

**Классы несбалансированы**

In [47]:
# lemmatizer = WordNetLemmatizer()

# def lemmatize_text(text):
#     text = text.lower()
#     lemm_text = ''.join(lemmatizer.lemmatize(text))
#     cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
#     return ' '.join(cleared_text.split())

# data['lemm_text'] = data['text'].apply(lemmatize_text)
# data = data.drop(['text'], axis=1)

# print(data)

In [48]:
data = data.drop(['Unnamed: 0'], axis=1)

# функиция очистки текста:

def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [49]:
%%time

# очистка постов:

data['text'] = data['text'].apply(clear_text) 

CPU times: user 4.14 s, sys: 32.9 ms, total: 4.18 s
Wall time: 4.2 s


In [50]:
# РОS-тэгирование

def pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,               #прилагательное
                "N": wordnet.NOUN,              #существительное
                "V": wordnet.VERB,              #глагол
                "R": wordnet.ADV                #наречие
               }  
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

# функция лемматизации текста:

def lemm_text(text):
    text = [lemmatizer.lemmatize(w, pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [51]:
%%time

nltk.download('averaged_perceptron_tagger')

#лемматизация текста:

data['text'] = data['text'].apply(lemm_text) 

# 19 мин

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


CPU times: user 17min 47s, sys: 1min 40s, total: 19min 28s
Wall time: 19min 29s


**ПРОВЕРКА: НАЧАЛО**

In [52]:
sentence1 = "The striped bats are hanging on their feet for best"
sentence2 = "you should be ashamed of yourself went worked"
df_my = pd.DataFrame([sentence1, sentence2], columns = ['text'])
print(df_my)

                                                text
0  The striped bats are hanging on their feet for...
1      you should be ashamed of yourself went worked


In [53]:
nlp = spacy.load('en_core_web_sm')

def func(text):
    doc = nlp(text) 
    return ' '.join([token.lemma_ for token in doc])

In [54]:
print(df_my['text'].apply(func))

0    the stripe bat be hang on their foot for good
1        you should be ashamed of yourself go work
Name: text, dtype: object


**ПРОВЕРКА: КОНЕЦ**

**по идее что-то типа такого должно работать (не хватает только удаления символов типа точек, запятых и тд), но оно грузится бесконечно долго, поэтому через POS-тэгирование сделал в итоге**

In [55]:
# data['lemm_text'] = data['text'].apply(func)

**Провели лемматизацию**

In [56]:
#data.head()

In [57]:
# Разбиваем датасет на выборки с пропорциями - train:test = 4:1

target = data['toxic']
features = data.drop(['toxic'], axis=1)

# features_train, features_valid, target_train, target_valid = train_test_split(features, 
#                                                                               target, 
#                                                                               test_size=0.5, 
#                                                                               random_state=777)
# features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
#                                                                             target_valid, 
#                                                                             test_size=0.5,
#                                                                             random_state=777)

features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.2, 
                                                                            random_state=777,
                                                                            stratify=target)

# nltk.download('stopwords')
# stpwrds = set(nltk_stopwords.words('english'))

# vectorizer = TfidfVectorizer(stop_words=stpwrds)

# features_train = vectorizer.fit_transform(features_train['text'])
# features_valid = vectorizer.transform(features_valid['lemm_text'].values.astype('U'))
# features_test = vectorizer.transform(features_test['text'])

for i in [features_train, target_train, features_test, target_test]:
    print(i.shape)

(127433, 1)
(127433,)
(31859, 1)
(31859,)


## Обучение

### Logistic Regression

In [58]:
# %%time

# model_LR = LogisticRegression(random_state=777)
# param_LR = [{'C':[10], 'class_weight': ['balanced']}]
# grid_LR = GridSearchCV(model_LR, param_LR, scoring = 'f1', cv = 3).fit(features_train, target_train)
# print('best params:')
# best_param_LR = grid_LR.best_params_
# print(best_param_LR)
# print('--------------------')
# print('grid score')
# means = grid_LR.cv_results_['mean_test_score']
# stds = grid_LR.cv_results_['std_test_score']
# for mean, std, params in zip(means, stds, grid_LR.cv_results_['params']):
#     print("%0.6f for %r"% (mean, params))
# print()

# f1_LR_cv = max(means)

In [ ]:
%%time

pipeline = Pipeline([('vect', TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ('lr', LogisticRegression())])
    
parameters = {'lr__solver': ('liblinear', 'saga', 'newton-cg', 'lbfgs'),
              'lr__C': (.1, 1, 5, 10),
              'lr__random_state': ([777]),
              'lr__max_iter': ([200]),
              'lr__class_weight': (['balanced'])} 
    
features_train = features_train.text    
    
grid_LR = GridSearchCV(pipeline, parameters, scoring = 'f1', cv = 3).fit(features_train, target_train)

means_lr = grid_LR.cv_results_['mean_test_score']
f1_lr_cv = max(means_lr)

print('F1 cv:', round(f1_lr_cv, 4))
print('best params:', grid_LR.best_params_)
print()

# 22 мин

In [ ]:
#classificator = LogisticRegression()
# model_LR.set_params(**best_param_LR).fit(features_train, target_train)
# pred_LR = model_LR.predict(features_valid)
# f1_LR_val = f1_score(target_valid, pred_LR)
# print('F1 на cv', f1_LR_cv)
# print('F1 на валидации', f1_LR_val)

### Decision Tree Classifier

In [ ]:
# %%time

# model_DTC = DecisionTreeClassifier()
# param_DTC = [{'max_depth':[x for x in range(50, 100, 2)], 'random_state':[777]}]
# grid_DTC = GridSearchCV(model_DTC, param_DTC, scoring = 'f1',cv = 3).fit(features_train, target_train)
# print("Best parameters set found on development set:")
# print()
# best_param_DTC = grid_DTC.best_params_
# print(best_param_DTC)
# print()
# print("Grid scores on development set:")
# print()
# means = grid_DTC.cv_results_['mean_test_score']
# stds = grid_DTC.cv_results_['std_test_score']
# for mean, std, params in zip(means, stds, grid_DTC.cv_results_['params']):
#     print("%0.6f for %r"% (mean, params))
# print()

# f1_DTC_cv = max(means)

# этот блок с кодом будет грузиться долго - мин 15-20, но он работает

In [ ]:
# %%time

# model_DTC = DecisionTreeClassifier()
# model_DTC.set_params(**best_param_DTC).fit(features_train, target_train)
# predict_DTC_val = model_DTC.predict(features_valid)
# f1_DTC_val = f1_score(target_valid, predict_DTC_val)
# print('F1 на cv', f1_DTC_cv)
# print('F1 на валидации', f1_DTC_val)

In [ ]:
%%time

pipeline = Pipeline([('vect', TfidfVectorizer(stop_words='english')), 
                     ('dtc', DecisionTreeClassifier())])
    
parameters = {'dtc__max_depth': ([x for x in range(1, 25)]),
              'dtc__random_state': ([777]), 
              'dtc__class_weight': (['balanced'])}

grid_DTC = GridSearchCV(pipeline, parameters, scoring= 'f1', cv = 3).fit(features_train, target_train)

means_dtc = grid_DTC.cv_results_['mean_test_score']
f1_dtc_cv = max(means_dtc)

print('F1 cv:', round(f1_dtc_cv, 4))
print('best params:', grid_DTC.best_params_)
print()

# 13 мин

### CatBoostClassifier

In [ ]:
# %%time

# model_CBC = CatBoostClassifier(verbose = False, iterations = 100).fit(features_train, target_train)
# predict_CBC_val = model_CBC.predict(features_valid)
# f1_CBC_cv = cross_val_score(model_CBC, features_train, target_train, cv = 3, scoring = 'f1').mean()
# f1_CBC_val = f1_score(target_valid, predict_CBC_val)
# print('F1 на cv', f1_CBC_cv)
# print('F1 на валидации', f1_CBC_val)

# 10 мин

In [ ]:
%%time

pipeline = Pipeline([('vect', TfidfVectorizer(stop_words='english')), 
                     ('cbc', CatBoostClassifier())])
    
parameters = {'cbc__verbose': ([False]),
              'cbc__iterations': ([100]),
              'cbc__random_state': ([777]),
              'cbc__class_weights':([(1, 1), (1, 11)])}

grid_CBC = GridSearchCV(pipeline, parameters, scoring= 'f1', cv = 3).fit(features_train, target_train)

means_cbc = grid_CBC.cv_results_['mean_test_score']
f1_cbc_cv = max(means_cbc)

print('F1 cv =', round(f1_cbc_cv, 4))
print('best params:', grid_CBC.best_params_)

# 28 мин

**Промежуточный результат**

In [ ]:
index = ['Logistic Regression',
         'Decision Tree',
         'CatBoost']
data = {'F1 cv':[f1_lr_cv, f1_dtc_cv, f1_cbc_cv]}

result = pd.DataFrame(data = data, index = index)
result['Выполнение задачи'] = result['F1 cv'] >= 0.75
result

**Модель логистической регрессии показала наилучший результат**

## Выводы

In [ ]:
# model_LR = LogisticRegression().set_params(**best_param_LR).fit(features_train, target_train)
# pred_LR_test = model_LR.predict(features_test)
# print('F1 test:', f1_score(target_test, pred_LR_test))

In [ ]:
pred_lr_test = grid_LR.predict(features_test.text)
f1_lr_test = f1_score(target_test, pred_lr_test)
print('F1 LR test:', round(f1_lr_test, 4))

In [ ]:
plt.figure(figsize=[12, 9])

plt.plot([0, 1], [0, 1], linestyle = '--', label = 'RandomModel')

prob_lr_test = grid_LR.predict_proba(features_test.text)[:, 1]
fpr, tpr, thresholds = roc_curve(target_test, prob_lr_test)

plt.plot(fpr, tpr, label = 'LogisticRegression')
print('Метрики LogisticRegression:')
print()
print('ROC AUC:', roc_auc_score(target_test, prob_lr_test))
print('F1:', f1_score(target_test, pred_lr_test))
print('Precision:', precision_score(target_test, pred_lr_test))
print('Recall:', recall_score(target_test, pred_lr_test))
print('Accuracy:', accuracy_score(target_test, pred_lr_test))

plt.xlim([0,1])
plt.ylim([0,1])

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.legend(loc = 'lower right', fontsize = 'x-large')

plt.title('ROC-кривая')
plt.show()

Создали три модели, способные осуществлять модерацию комментариев (задача классификации) - логистическая регрессия, дерево решений и CatBoost. 

Данные разбили на две выборки - тренировочную и тестовую.

Метрика оценивания качества - F1-мера, сочетающая в себе Accuracy и Recall. По условию задачу значение данной метрики должно составить не менее 0.75.

F1 составило: 0.75... для модели Logistic Regression, 0.6353 для Decision Tree Classifier и 0.7385 для CatBoost Classifier. Таким образом, согласно условию задачи, нам подходит модель логистической регрессии, которую мы посмотрели на тестовой выборке - значение F1 составило 0.7563.

Логистическая регрессия показала наилучший результат (>0.75), следовательно, для данной задачи выбираем эту модель.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны